# Observability Integration Tests - Notebook API

This notebook tests observability using the notebook interface that mirrors CLI commands.

## Setup
Configure your agent ID and session ID:

In [ ]:
from bedrock_agentcore_starter_toolkit import Observability

# Test configuration - UPDATE THESE VALUES
TEST_AGENT_ID = "test_eval_1-Ux9OE986P4"  # Replace with your agent ID
TEST_SESSION_ID = "cc8a8e69-8bed-4e5f-9a06-9a58550fd713"  # Replace with your session ID
TEST_REGION = "us-east-1"  # Update with your AWS region

print("✅ Configuration:")
print(f"  Agent ID: {TEST_AGENT_ID}")
print(f"  Session ID: {TEST_SESSION_ID}")
print(f"  Region: {TEST_REGION}")

ImportError: cannot import name 'Observability' from 'bedrock_agentcore_starter_toolkit.notebook' (/Users/vivekbh/workspaces/agentcore/bedrock-agentcore-starter-toolkit/src/bedrock_agentcore_starter_toolkit/notebook/__init__.py)

## Test 1: Initialize Observability

Create observability instance with agent_id.

In [ ]:
# Initialize with explicit agent_id
obs = Observability(agent_id=TEST_AGENT_ID, region=TEST_REGION)

print("✅ Test 1 PASSED: Observability initialized")

## Test 2: obs.list() - Basic Session Listing

List all traces in a session (equivalent to `agentcore obs list`).

In [ ]:
# List traces from specific session
trace_data = obs.list(session_id=TEST_SESSION_ID)

print(f"\n✅ Test 2 PASSED: Listed {len(trace_data.traces)} traces")

## Test 3: obs.list() - Latest Session Discovery

List traces without session_id (auto-discovers latest).

In [ ]:
# List traces from latest session
trace_data = obs.list()

print(f"\n✅ Test 3 PASSED: Auto-discovered and listed {len(trace_data.traces)} traces")

## Test 4: obs.list() - Error Filtering

List only traces with errors.

In [ ]:
# List only failed traces
trace_data = obs.list(session_id=TEST_SESSION_ID, errors=True)

print(f"\n✅ Test 4 PASSED: Found {len(trace_data.traces)} failed trace(s)")

## Test 5: obs.show() - Latest Trace (Default)

Show latest trace from session (equivalent to `agentcore obs show`).

In [ ]:
# Show latest trace
trace_data = obs.show(session_id=TEST_SESSION_ID)

print("\n✅ Test 5 PASSED: Showed latest trace")

## Test 6: obs.show() - Nth Most Recent Trace

Show 2nd most recent trace using last parameter.

In [ ]:
# Show 2nd most recent trace
trace_data = obs.show(session_id=TEST_SESSION_ID, last=2)

print("\n✅ Test 6 PASSED: Showed 2nd most recent trace")

## Test 7: obs.show() - All Traces

Show all traces in session with full details.

In [ ]:
# Show all traces with full details
trace_data = obs.show(session_id=TEST_SESSION_ID, all=True)

print(f"\n✅ Test 7 PASSED: Showed all {len(trace_data.traces)} traces")

## Test 8: obs.show() - All Traces with Verbose

Show all traces with full payloads (no truncation).

In [ ]:
# Show all traces with verbose mode
trace_data = obs.show(session_id=TEST_SESSION_ID, all=True, verbose=True)

print("\n✅ Test 8 PASSED: Showed all traces with verbose output")

## Test 9: obs.show() - Error Traces Only

Show only failed traces in session.

In [ ]:
# Show only failed traces
trace_data = obs.show(session_id=TEST_SESSION_ID, errors=True)

print("\n✅ Test 9 PASSED: Showed error traces")

## Test 10: Auto-discover Trace ID

Extract a trace ID from the session for specific trace testing.

In [ ]:
# Get trace data and extract first trace ID
trace_data = obs.list(session_id=TEST_SESSION_ID)
trace_ids = list(trace_data.traces.keys())

if trace_ids:
    TEST_TRACE_ID = trace_ids[0]
    print("✅ Test 10 PASSED: Discovered trace ID")
    print(f"Trace ID: {TEST_TRACE_ID}")
else:
    TEST_TRACE_ID = None
    print("⚠️  Test 10 SKIPPED: No traces found")

## Test 11: obs.show() - Specific Trace

Show a specific trace by trace_id.

In [ ]:
if TEST_TRACE_ID:
    # Show specific trace
    trace_data = obs.show(trace_id=TEST_TRACE_ID)

    print("\n✅ Test 11 PASSED: Showed specific trace")
else:
    print("⚠️  Test 11 SKIPPED: No trace ID available")

## Test 12: obs.show() - Specific Trace with Verbose

Show specific trace with full payloads.

In [ ]:
if TEST_TRACE_ID:
    # Show specific trace with verbose
    trace_data = obs.show(trace_id=TEST_TRACE_ID, verbose=True)

    print("\n✅ Test 12 PASSED: Showed specific trace with verbose")
else:
    print("⚠️  Test 12 SKIPPED: No trace ID available")

## Test 13: obs.show() - Export to JSON

Export trace data to JSON file.

In [ ]:
if TEST_TRACE_ID:
    output_file = "/tmp/test_trace_export.json"

    # Show and export trace
    trace_data = obs.show(trace_id=TEST_TRACE_ID, output=output_file)

    # Verify file exists
    from pathlib import Path

    assert Path(output_file).exists(), "Output file not created"

    print(f"\n✅ Test 13 PASSED: Exported trace to {output_file}")
else:
    print("⚠️  Test 13 SKIPPED: No trace ID available")

## Test 14: Initialize with Agent Name

Initialize observability using agent name from config.

In [ ]:
# Try initializing with agent name (requires .bedrock_agentcore.yaml)
try:
    obs_named = Observability(agent_name="test_eval_1")
    print("✅ Test 14 PASSED: Initialized with agent name from config")
except Exception as e:
    print(f"⚠️  Test 14 SKIPPED: Config not found or agent name not in config ({e})")

## Test 15: Auto-detect Latest Session

Test session auto-discovery without providing session_id.

In [ ]:
# Show latest trace from auto-discovered session
trace_data = obs.show()

print("\n✅ Test 15 PASSED: Auto-discovered session and showed trace")

## Test 16: Error Handling - Conflicting Parameters

Test that conflicting parameters raise appropriate errors.

In [ ]:
# Test 1: trace_id and session_id together (should fail)
try:
    obs.show(trace_id=TEST_TRACE_ID, session_id=TEST_SESSION_ID)
    print("❌ Test 16a FAILED: Should have raised error")
except ValueError:
    print("✅ Test 16a PASSED: Correctly rejected conflicting parameters")

# Test 2: trace_id with all (should fail)
try:
    obs.show(trace_id=TEST_TRACE_ID, all=True)
    print("❌ Test 16b FAILED: Should have raised error")
except ValueError:
    print("✅ Test 16b PASSED: Correctly rejected conflicting parameters")

# Test 3: all with last (should fail)
try:
    obs.show(session_id=TEST_SESSION_ID, all=True, last=2)
    print("❌ Test 16c FAILED: Should have raised error")
except ValueError:
    print("✅ Test 16c PASSED: Correctly rejected conflicting parameters")

## Test 17: Custom Time Range

Test querying with custom lookback period.

In [ ]:
# Query with 1 day lookback
trace_data = obs.list(session_id=TEST_SESSION_ID, days=1)

print("\n✅ Test 17 PASSED: Queried with custom time range (1 day)")

## Test 18: TraceData Properties

Test accessing trace data properties and methods.

In [ ]:
trace_data = obs.list(session_id=TEST_SESSION_ID)

print(f"Session ID: {trace_data.session_id}")
print(f"Total traces: {len(trace_data.traces)}")
print(f"Total spans: {len(trace_data.spans)}")
print(f"Runtime logs: {len(trace_data.runtime_logs)}")

# Test utility methods
for trace_id in list(trace_data.traces.keys())[:1]:
    spans = trace_data.traces[trace_id]
    duration = trace_data.calculate_trace_duration(spans)
    error_count = trace_data.count_error_spans(spans)
    input_text, output_text = trace_data.get_trace_messages(trace_id)

    print(f"\nTrace {trace_id[:16]}...")
    print(f"  Duration: {duration:.2f}ms")
    print(f"  Errors: {error_count}")
    print(f"  Input: {input_text[:50] if input_text else 'N/A'}...")
    print(f"  Output: {output_text[:50] if output_text else 'N/A'}...")

print("\n✅ Test 18 PASSED: TraceData properties and methods work")

## Test 19: Filter Error Traces

Test filtering functionality.

In [ ]:
trace_data = obs.list(session_id=TEST_SESSION_ID)

# Filter to error traces
error_traces = trace_data.filter_error_traces()

print(f"Total traces: {len(trace_data.traces)}")
print(f"Error traces: {len(error_traces)}")

print("\n✅ Test 19 PASSED: Error filtering works")

## Summary

Display test results summary.

In [ ]:
print("\n" + "=" * 80)
print("🎉 INTEGRATION TEST SUITE COMPLETE")
print("=" * 80)

print("\nTested Notebook API Commands:")
print("  ✅ obs.list(session_id)          → List all traces in session")
print("  ✅ obs.list()                    → Auto-discover latest session")
print("  ✅ obs.list(errors=True)         → Filter error traces")
print("  ✅ obs.show(session_id)          → Show latest trace")
print("  ✅ obs.show(session_id, last=N)  → Show Nth trace")
print("  ✅ obs.show(session_id, all=True)→ Show all traces")
print("  ✅ obs.show(verbose=True)        → Show full payloads")
print("  ✅ obs.show(trace_id)            → Show specific trace")
print("  ✅ obs.show(output='file.json')  → Export to JSON")
print("  ✅ Error handling                → Parameter validation")
print("  ✅ TraceData methods             → Filtering, messages, calculations")

print("\nConfiguration:")
print(f"  Agent ID: {TEST_AGENT_ID}")
print(f"  Session ID: {TEST_SESSION_ID}")
print(f"  Trace ID: {TEST_TRACE_ID if 'TEST_TRACE_ID' in dir() else 'N/A'}")
print(f"  Region: {TEST_REGION}")

print("\n💡 API matches CLI commands:")
print("  CLI: agentcore obs list --session-id abc123")
print("  API: obs.list(session_id='abc123')")
print("")
print("  CLI: agentcore obs show --session-id abc123 --all --verbose")
print("  API: obs.show(session_id='abc123', all=True, verbose=True)")